In [ ]:
!pip install -q gradio_client gradio torch torchaudio transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires opena

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

processor = BlipProcessor.from_pretrained("jaimik69/blip_finetuned")
model = BlipForQuestionAnswering.from_pretrained("jaimik69/blip_finetuned")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch

def transcribe(audio, img_url):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    prompt = transcriber({"sampling_rate": sr, "raw": y})["text"]
    raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
    inputs = processor(raw_image, text=prompt, return_tensors="pt")
    generated_ids = model.generate(**inputs, max_new_tokens=10)
    aqa_ans = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    return prompt, aqa_ans, gr.Image(img_url)

demo = gr.Interface(
    transcribe,
    [gr.Audio(sources=["microphone"], label = "User Audio Input"), gr.Textbox(label="User Image URL")],
    [gr.Textbox(label="Question"), gr.Textbox(label="Answer"), gr.Image(label = "User Image")],
    title = 'Vox Helios', theme = 'dark-grass',
    description = 'An Audio Question Answering Project'
)

if __name__ == "__main__":
    demo.launch(debug=True,auth=("sai", "letmein"))

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:528: UserWarning: Cannot load dark-grass. Caught Exception: The space dark-grass does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://90c36f56b801550f7e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://90c36f56b801550f7e.gradio.live
